# Import libraries

In [266]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import time
import json

# url with city name

In [267]:
url = "https://me.cleartrip.com/hotels/united-states/miami/"

# Webdriver

In [268]:
driver = webdriver.Firefox()
driver.get(url)

# Get number of pages

In [269]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
num_pages = int(soup.select('div.pagination a')[-2].text)

# Extract all hotel links

In [270]:
hotel_list = [] # to store all links

In [271]:
for i in range(num_pages):
    # get current page
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # extract and store links one by one for the current page
    j=0
    while True:
        try:
            hotel_link = 'https://me.cleartrip.com/hotels/info/' + \
                          soup.find_all('div', attrs={'class':"hotels-thump"})[j].find('a').get('data-hotelid').split(',')[0]
            hotel_list.append(hotel_link)
            j=j+1
            time.sleep(2)
        except Exception as e:
            break
    # click on next element to go to next page
    try:
        driver.find_element_by_css_selector('a.next_page').click()
    except Exception as e:
        break
    time.sleep(5)

Message: Unable to locate element: a.next_page



In [272]:
# function to get hotel information
def get_info(url):
    all_info = {} # dictionary for the hotel information
    hotel_info = {} # to store 'about the hotel' part information
    # use requests and beautifulsoup to collect the hotel info webpage
    page = requests.get(url) 
    soup = BeautifulSoup(page.text, 'html.parser')
    # name
    hotel_name = soup.find_all('h1', attrs={'class':"hotel-title"})[0].get_text()
    all_info.update({'hotel_name':hotel_name})
    # url
    all_info.update({'hotel_url':url})
    # address
    try:
        hotel_address = soup.find_all('div', attrs={'class':"hotels-location"})[0].get_text()
        all_info.update({'hotel_address':hotel_address})
    except Exception as e:
        pass
    # rating
    try:
        b = soup.find_all('div', attrs={'class':"hotelTaRating"})[1].find_all('span',attrs={'class':"schema-TAcount"})[0].get_text()
        rating = float(b.split('/')[0])
        all_info.update({'hotel_rating':rating})
    except Exception as e:
        pass
    # review num
    try:
        b = soup.find_all('div', attrs={'class':"hotelTaRating"})[1].find_all('span',attrs={'itemprop':"reviewCount"})[0].get_text()
        review_num = int(b)
        all_info.update({'hotel_review_num':review_num})
    except Exception as e:
        pass
    # price
    try:
        b = soup.find_all('p', attrs={'class':"minprice"})[0].get_text().strip()
        price = int(b)
        all_info.update({'hotel_price':price})
    except Exception as e:
        pass
    # about the hotel info part
    # when more paragraphs present]
    try:
        k = soup.find_all('div', attrs={'class':"hotel-description"})[0]
        for i in range(len(k)):
            hotel_info.update({k.find_all('p')[i].get_text().split(': ')[0]:k.find_all('p')[i].get_text().split(': ')[1]})
    except Exception as e:
        # when one paragraph present
        try:
            k = soup.find_all('div', attrs={'class':"hotel-description"})[0]
            hotel_info = k.get_text()
        except Exception as e:
            pass
    
    #update all_info
    all_info.update({'hotel_info':hotel_info})
    # check in check out and rooms
    try:
        all_info.update({soup.find_all('ul', attrs={'class':"list-inline hotel-rules"})[0].find_all('li')[0].get_text().split(' ')[2]\
                     :soup.find_all('ul', attrs={'class':"list-inline hotel-rules"})[0].find_all('li')[0].get_text().split(' ')[1],\
         soup.find_all('ul', attrs={'class':"list-inline hotel-rules"})[0].find_all('li')[1].get_text().split(' ')[2]\
         :soup.find_all('ul', attrs={'class':"list-inline hotel-rules"})[0].find_all('li')[1].get_text().split(' ')[1],
         soup.find_all('ul', attrs={'class':"list-inline hotel-rules"})[0].find_all('li')[2].get_text().split(' ')[2]\
         :soup.find_all('ul', attrs={'class':"list-inline hotel-rules"})[0].find_all('li')[2].get_text().split(' ')[1]})
    except Exception as e:
        pass
    # other info
    i=1
    while True:
        try:
            all_info.update({soup.find_all('div', attrs={'class':"amenities-title"})[i].get_text():\
                             soup.find_all('ul', attrs={'class':"list-inline amenities"})[i-1].get_text().strip().split('  ')})
            i=i+1
        except Exception as e:
            break   
    return all_info

# one particular hotel info

In [284]:
# to return information about a particular hotel in json format
# url = 'url here'
# get_info(url)

# store info about all hotels

In [279]:
all_hotels_info = [] # list to store info about all hotels

In [280]:
# finally store info about all hotels
cnt = 0
for i in hotel_list:
    all_hotels_info.append(get_info(i))
    # tracking if an eror occurs
    print(i)
    cnt+=1
    print(cnt)
    time.sleep(0.5)
    

https://me.cleartrip.com/hotels/info/192017
1
https://me.cleartrip.com/hotels/info/292714
2
https://me.cleartrip.com/hotels/info/345087
3
https://me.cleartrip.com/hotels/info/367313
4
https://me.cleartrip.com/hotels/info/367317
5
https://me.cleartrip.com/hotels/info/367353
6
https://me.cleartrip.com/hotels/info/453735
7
https://me.cleartrip.com/hotels/info/514188
8
https://me.cleartrip.com/hotels/info/275840
9
https://me.cleartrip.com/hotels/info/338669
10
https://me.cleartrip.com/hotels/info/345107
11
https://me.cleartrip.com/hotels/info/531425
12
https://me.cleartrip.com/hotels/info/569703
13
https://me.cleartrip.com/hotels/info/660995
14
https://me.cleartrip.com/hotels/info/1214040
15
https://me.cleartrip.com/hotels/info/1232342
16
https://me.cleartrip.com/hotels/info/192045
17
https://me.cleartrip.com/hotels/info/192008
18
https://me.cleartrip.com/hotels/info/253754
19
https://me.cleartrip.com/hotels/info/417953
20
https://me.cleartrip.com/hotels/info/140302
21
https://me.cleartrip

172
https://me.cleartrip.com/hotels/info/627310
173
https://me.cleartrip.com/hotels/info/677090
174
https://me.cleartrip.com/hotels/info/773434
175
https://me.cleartrip.com/hotels/info/777588
176
https://me.cleartrip.com/hotels/info/1224846
177
https://me.cleartrip.com/hotels/info/2310394
178
https://me.cleartrip.com/hotels/info/2315214
179
https://me.cleartrip.com/hotels/info/2314762
180
https://me.cleartrip.com/hotels/info/2321218
181
https://me.cleartrip.com/hotels/info/2321128
182
https://me.cleartrip.com/hotels/info/2322576
183
https://me.cleartrip.com/hotels/info/2323126
184
https://me.cleartrip.com/hotels/info/1259022
185
https://me.cleartrip.com/hotels/info/1282014
186
https://me.cleartrip.com/hotels/info/1297216
187
https://me.cleartrip.com/hotels/info/139365
188
https://me.cleartrip.com/hotels/info/140206
189
https://me.cleartrip.com/hotels/info/140201
190
https://me.cleartrip.com/hotels/info/140200
191
https://me.cleartrip.com/hotels/info/140203
192
https://me.cleartrip.com/

KeyboardInterrupt: 

In [286]:
all_hotels_info # scrapped till 319 hotels

[{'hotel_name': 'Hilton Bentley South Beach',
  'hotel_url': 'https://me.cleartrip.com/hotels/info/192017',
  'hotel_rating': 4.0,
  'hotel_review_num': 1103,
  'hotel_info': 'Deluxe Class Modern Hotel. Beachfront All Suite property on Ocean Drive, in the world-class environment of South Beach in the exclusive neighborhood of "Sofi(South of Fifth Street). Minutes from Coral Gables, Coconut Grove and Miami\'s international business district on Brickell Avenue. Miami International Airport is 15 minutes away.Oversized junior suites (350 sq. ft.) furnished with 1 King Bed, custom designed with European furnishings, with floor-to-ceiling windows revealing views of the city or ocean, and are equipped with kitchenette, luxurious oversized marble bathroom and state of the art amenities.Upscale Italian restaurant on site and numerous dining options within the surrounding area offering a variety of tastes.European-style boutique hotel in two 10-story towers.Intimate and elegantly decorated lobby

In [288]:
# store the list to file
with open('all_hotels_info.txt', 'w') as json_list:  
    json.dump(all_hotels_info, json_list)